In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.9 MB/s eta 0:00:00


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
device

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


device(type='cuda')

## Model

Load [ruGPT3](https://huggingface.co/ai-forever/rugpt3large_based_on_gpt2).

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1536, out_features=50257, bias=False)
)

# Zero-shot

We use model loss for the zero-shot classification.

**Idea:** choose the answer which corresponds to the variant with the lowest.


In [ ]:
import math
def calc_loss(phrase: str,
                        tokenizer,
                        model):

    phrase = tokenizer.encode(phrase)
    # Если длина фразы 1 токен, то дальше ошибка вылезет :(
    if len(phrase) == 1:
         phrase.append(tokenizer.eos_token_id)
    phrase = torch.tensor(phrase, dtype=torch.long, device=device)
    phrase = phrase.unsqueeze(0)  # .repeat(num_samples, 1)
    with torch.no_grad():
        loss = model(phrase, labels=phrase)

    loss[0].item()


    return loss[0].item()

def get_loss_num(text):
    loss = calc_loss(phrase=text, model=model, tokenizer=tokenizer)
    return loss

def clean(text):
    text = re.sub(r'\((\d+)\)', '', text)
    return text

### Twitter: tone analysis

In [ ]:
text = 'жизнь отличная'
get_loss_num('Позитивный твит: ' + text + ')))')

6.151878356933594

In [ ]:
get_loss_num('Негативный твит: ' + text + '(((')

7.050114154815674

In [ ]:
def predict_zero_shot(text):
  pos_loss = get_loss_num('Позитивный твит: ' + text + ')))')
  neg_loss = get_loss_num('Негативный твит: ' + text + '(((')
  if pos_loss < neg_loss:
    return 'positive'
  return 'negative'

predict_zero_shot(text)

'positive'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls 'gdrive/MyDrive/Colab Notebooks/data'

In [ ]:
import pandas as pd
df = pd.read_csv('gdrive/MyDrive/Colab Notebooks/data/twitter_short.csv', index_col = 0)
df.head()

In [11]:
df.tail()

,text,label
95,"Встречайте, мои супер одногруппницы, будущие и...",positive
96,"все,я вас покидаю,результаты гляну вечером)#би...",positive
97,RT @Dasha_crazy_69: @DashkaTeddy дыы))) но кто...,positive
98,Почти приехали в родное селенье!) @ москва-рига,positive
99,На*уй ваши Канары и Мальдивы ! Тут новая тема ...,positive


In [12]:
from sklearn.metrics import accuracy_score
df['preds'] = df.text.apply(predict_zero_shot)
accuracy_score(df.label, df.preds)

0.74

# home work

In [25]:
acc_list = []

In [46]:
def predict_zero_shot(text,pos_prmt,neg_prmt):
  pos_loss = get_loss_num(pos_prmt[0] + text + pos_prmt[1])
  neg_loss = get_loss_num(neg_prmt[0] + text + neg_prmt[1])
  if pos_loss < neg_loss:
    return 'positive'
  return 'negative'

pos_prmt=['Позитивный твит: ', ')))']
neg_prmt=['Негативный твит: ', '(((']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.74

In [47]:
pos_prmt=['Позитивный: ', ')))']
neg_prmt=['Негативный: ', '(((']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.78

In [48]:
pos_prmt=['Позитивный: ', '']
neg_prmt=['Негативный: ', '']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.52

In [49]:
pos_prmt=['положительный текст: ', ')))']
neg_prmt=['отрицательный текст: ', '(((']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.89

In [50]:
pos_prmt=['положительный позитивный: ', ')))']
neg_prmt=['отрицательный негативный: ', '(((']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.78

In [51]:
pos_prmt=['положительный позитивный текст: ', ')))']
neg_prmt=['отрицательный негативный текст: ', '(((']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.89

In [52]:
pos_prmt=['положительный текст: ', '']
neg_prmt=['отрицательный текст: ', '']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.5

In [54]:
pos_prmt=['положительный текст: ', '>>>']
neg_prmt=['отрицательный текст: ', '<<<']
df['preds'] = df.text.apply(lambda x: predict_zero_shot(x, pos_prmt, neg_prmt))
cur_acc = accuracy_score(df.label, df.preds)
acc_list.append(["' + ______ + '".join(pos_prmt), "' + ______ + '".join(neg_prmt), cur_acc])
cur_acc

0.61

## вывод

In [65]:
pd.DataFrame(acc_list, columns=['positive_promt', 'negative_promt', 'accuracy'])

,positive_promt,negative_promt,accuracy
0,Позитивный твит: ' + ______ + '))),Негативный твит: ' + ______ + '(((,0.74
1,Позитивный: ' + ______ + '))),Негативный: ' + ______ + '(((,0.78
2,Позитивный: ' + ______ + ',Негативный: ' + ______ + ',0.52
3,положительный текст: ' + ______ + '))),отрицательный текст: ' + ______ + '(((,0.89
4,положительный позитивный: ' + ______ + '))),отрицательный негативный: ' + ______ + '(((,0.78
5,положительный позитивный текст: ' + ______ + '))),отрицательный негативный текст: ' + ______ + '(((,0.89
6,положительный текст: ' + ______ + ',отрицательный текст: ' + ______ + ',0.50
7,положительный текст: ' + ______ + '>>>,отрицательный текст: ' + ______ + '<<<,0.61


лучший промт (скор = 0.89, плюс более короткий): </br>
'положительный текст: ' + text + ')))' </br>
'отрицательный текст: ' + text + '((('